In [ ]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast
from fuzzywuzzy import process
from nltk.corpus import words

## Initial Experiments

In [ ]:
total = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\total_summary_stats_updated.csv")

In [ ]:
patient_id = total["patient_id"].to_list()
patient_id = patient_id[:10]

In [ ]:
matches = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_matches.csv")

In [ ]:
# ONLY RUN ONCE #

# vertifying that all match records can be found
for i in range(len(patient_id)):
    specific_matches = matches[matches["PatientID"] == patient_id[i]]
    if (len(specific_matches) <= 0):
        print("NOT FOUND: ", patient_id[i])
    #specific_matches.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Specific_Matches\patient_{}.csv".format(i+1))

In [ ]:
p1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Specific_Matches\patient_1.csv")

In [ ]:
unique_ids = matches["PatientID"].unique().tolist()
print(len(unique_ids))

In [ ]:
# interesting match cases
# patient_2 in specific study row 6, cream that has mci inside it

In [ ]:
print(subset_loc_clone)

## Final Code

In [ ]:
p1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\3\apoe_3_matches_complete.csv")

In [ ]:
len(p1)

In [ ]:
p1.head()

In [ ]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\Sentence-Analysis\EDA\Getting_Data\keywords.csv")

In [ ]:
regex

In [ ]:
len(p1)

In [ ]:
# p = re.compile(r"(?i:)\bmemory\b|(?i:)\bno\b|(?-i:)\bOriented\b|(?-i:)\bMCI\b")

# p2 = re.compile(r"(?i:)\bMOCA\b")
# regexes = [p, p2]
# p_combined = re.compile(r'|'.join(x.pattern for x in regexes))
# print(p_combined)
# print(p)

k = regex["REGEX"].to_list()
c = regex["CASE"].to_list()
p_list = []

for i in range(len(k)):
    if (c[i] == 0):
        p_list.append(re.compile(k[i][5:], re.IGNORECASE))
    elif (c[i] == 1):
        p_list.append(re.compile(k[i]))
print(p_list)
p_all = (r"|".join(x.pattern for x in p_list))
p_all

In [ ]:
string = str(p1["merged_regex_sent"][5]) + " mCi."

print(string)
print(re.compile(r"\bmemory\b", re.IGNORECASE))

for p in p_list:
    for match in re.finditer(p, string):
        s = match.start()
        e = match.end()
        print ('String match "%s" at %d:%d' % (string[s:e], s, e))

In [ ]:
locations = []

for r in tqdm(p1['merged_regex_sent']):
    l = []
    for p in (p_list):
        for match in re.finditer(p, r):
            l.append(match.span())
    #print(l)
    locations.append(l)
        
p1["sequence_level_regex_location"] = locations

In [ ]:
len(p1)

In [ ]:
p1.head()

In [ ]:
p1["merged_regex_sent"][5]

In [ ]:
p2 = p1[p1["sequence_level_regex_location"].str.len() != 0]
len(p2)

In [ ]:
p1.head()

In [ ]:
p2.head()

In [ ]:
total_merged = []

for index, row in p1.iterrows():
    curr = str(row["regex_location"])
    curr = curr[1:len(curr) - 1]
    curr = list((eval(curr)))
    #print("CURR: ", len(curr))
    # print(curr)
    merged = []

    start = 0
    end = 0
    length = 0
    
    if (len(curr) == 2 and isinstance(curr[1], int)):
        start = curr[0]
        end = curr[1]
        #print(start, end)
        length = len(row["NoteTXT"])
        if (start < 100):
            start = 100
        elif (end > length - 100):
            end = length - 100
        curr = [(start - 100, end + 100)]
        
    else:
        loc_list = []
        for j in range(len(curr)):
            start = curr[j][0]
            end = curr[j][1]
            loc_list.append((start, end))
        length = len(row["NoteTXT"])
        for k in range(len(loc_list)):
            start = loc_list[k][0]
            end = loc_list[k][1]
            if (start < 100):
                start = 100
            elif (end > length - 100):
                end = length - 100
            loc_list[k] = (start - 100, end + 100)
        curr = loc_list
        
    for interval in curr:
        if not merged:
            merged.append(interval)
            continue

        prevInterval = merged.pop()
        if prevInterval[0] <= interval[0] <= prevInterval[1]:
            startTime = prevInterval[0]
            endTime = prevInterval[1] if prevInterval[1] > interval[1] else interval[1]
            merged.append((startTime, endTime))
        else:
            merged.append(prevInterval)
            merged.append(interval)
            
    total_merged.append(merged)
p1["merged_row_location"] = total_merged

In [ ]:
p1

In [ ]:
total_merged_note_txt = []
for index, row in p1.iterrows():
    curr = row["merged_row_location"]
    merged_note_txt = ""
    
    for i in range(len(curr)):
        start = curr[i][0]
        end = curr[i][1]
        
        merged_note_txt += " ------- " + row["NoteTXT"][start:end]
    total_merged_note_txt.append(merged_note_txt)
    
p1["merged_regex_sent"] = total_merged_note_txt

In [ ]:
len(p1)

In [ ]:
p1['merged_regex_sent'] = p1['merged_regex_sent'].str.split(' ',1).str[1].str.rpartition(' ')[0]

In [ ]:
#p1['NoteTXT'] = p1['NoteTXT'].astype(str)
p1 = p1.drop_duplicates(subset='merged_regex_sent', keep="first")
p1 = p1.replace(r'^\s*$', np.NaN, regex=True)
p1.reset_index(drop = True)

In [ ]:
p1.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\4\apoe_4_matches_updated_pruned.csv", index = False)

## Merging Regex matches 

In [ ]:
seq = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\5\apoe_5_matches_updated_pruned.csv")

In [ ]:
l = []
for note in tqdm(p2["merged_regex_sent"]):
    curr = []
    for p in (p_list):
        m = list(set(re.findall(p, note)))
        m = list(set(map(str.lower, m)))
        if (m != []):
            curr.append("".join(m))
    #print(curr)
    #print(l)
    l.append(str(curr))

p2["sequence_level_regex_match"] = l

In [ ]:
p2.head()

In [ ]:
len(p1[p1["merged_regex_match"] == "[\' mci\']"])

In [ ]:
len(p2[p2["sequence_level_regex_match"] == "[\'mci\']"])

In [ ]:
p2_snippet = p2[:500]
p2_snippet.to_excel(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\3\apoe_3_matches_7_11_snippet.xlsx", index = False)

In [ ]:
p2.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\5\apoe_5_matches_7_11.csv", index = False)

## Padding Each Sequence so that it becomes min 800 chars

In [ ]:
p2['char_count'] = p2['merged_regex_sent'].str.len()

In [ ]:
p2['note_char_count'] = p2['NoteTXT'].str.len()

In [ ]:
len(p2[p2['note_char_count'] < 800])

In [ ]:
p2.head()

In [ ]:
no_padding = p2[p2["char_count"] > 798] #798 b/c that is min cutoff to addd 1 char to each side
len(no_padding)

In [ ]:
need_padding = p2[p2["char_count"] <= 798]
len(need_padding)

In [ ]:
48359 +8411

In [ ]:
p2["merged_row_location"][0]

In [ ]:
need_padding.head()

In [ ]:
# getting revised intervals with padding accounted for

total_padded_intervals = []
for index, row in tqdm(need_padding.iterrows()):
    curr = str(row["merged_row_location"])
    curr = curr[1:len(curr) - 1]
    curr = list((eval(curr)))
    
    curr_char_count = row["char_count"]
    padded_len = int((800 - curr_char_count) / 2)
    
    if len(curr) == 2 and isinstance(curr[1], int):
        start = curr[0]
        end = curr[1]
        #print("b4", start, end)

        length = len(row["NoteTXT"])
        #print("length: ", length)
        if (start - padded_len < 0):
            start = 0

        if (end + padded_len > length):
            #print("reached")
            end = length - 1
            diff = 800 - (end - start)
            if (start - diff < 0):
                start = 0
            else:
                start -= diff

        if (start - padded_len > 0 and end + padded_len < length):
            start -= padded_len
            end += padded_len

        if (end - start < 750):
            curr_length = end - start
            add_to_end = 750 - curr_length
            if (end + add_to_end >= length):
                end = length  - 1
            else:
                end += add_to_end

        curr = [(start, end)]
    else:
        start = curr[0]
        end = curr[len(curr) - 1]
        length = len(row["NoteTXT"])
        if (start[0] - padded_len < 0):
            curr[0] = (0, curr[0][1])
        else:
            curr[0] = (curr[0][0] - padded_len, curr[0][1])
        if (end[1] + padded_len > length):
            curr[len(curr) - 1] = (curr[len(curr) - 1][0], length - 1)
        else:
            curr[len(curr) - 1] = (curr[len(curr) - 1][0], curr[len(curr) - 1][1] + padded_len)

        
#         curr[0] = (curr[0][0] - padded_len, curr[0][1])
#         curr[len(curr) - 1] = (curr[len(curr) - 1][0], curr[len(curr) - 1][1] + padded_len)
    
    #row["padded_merged_row_location"] = curr
    total_padded_intervals.append(curr)

need_padding["padded_merged_row_location"] = total_padded_intervals

In [ ]:
need_padding.head()

In [ ]:
total_merged_note_txt = []
for index, row in tqdm(need_padding.iterrows()):
    curr = str(row["padded_merged_row_location"])
    curr = curr[1:len(curr) - 1]
    curr = list((eval(curr)))
    #print(len(curr))
    
    merged_note_txt = ""
    
    if (len(curr) == 2 and isinstance(curr[1], int)):
        start = int(curr[0])
        end = int(curr[1])
        
        merged_note_txt += row["NoteTXT"][start:end]
        
    else:
        for i in range(len(curr)):
            start = curr[i][0]
            end = curr[i][1]
            if (start == 0):
                merged_note_txt += row["NoteTXT"][start:end]
            else:
                merged_note_txt += " ------- " + row["NoteTXT"][start:end]

        
    total_merged_note_txt.append(merged_note_txt)
    
need_padding["padded_merged_regex_sent"] = total_merged_note_txt

In [ ]:
need_padding.head()

In [ ]:
need_padding['padded_char_count'] = need_padding['padded_merged_regex_sent'].str.len()

In [ ]:
no_padding.columns

In [ ]:
need_padding.columns

In [ ]:
small = need_padding[need_padding['padded_char_count'] < 700] 
small

In [ ]:
len(need_padding)

In [ ]:
need_padding.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\3\Padded_Matches\apoe_3_padded_matches.csv")

In [ ]:
no_padding.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\3\Padded_Matches\apoe_3_not_padded_matches.csv")

In [ ]:
checking = need_padding[:250]
checking.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Padded_Matches\apoe_1_checking.csv")

## Edge Case Filtering (MCI, CDR, MMSE, MOCA)

In [ ]:
pattern_mci = re.compile(r'(?-i:)\bMCI\b|(?-i:)\bmci\b')
mci_matches = []
for note in tqdm(p2["merged_regex_sent"]):
    try:
        m = list(set(re.findall(pattern_mci, note)))
        m = list(set(m))
        mci_matches.append(str(m))
    except:
        mci_matches.append([])
p2["contains_mci"] = mci_matches

In [ ]:
string_2 = "------- : TC-99m Sestamibi According to standard departmental protocol, the patient was injected with 10.97 mCi of TC-99m Sestamibi IV at rest and ungated SPECT myocardial images were obtained. Subsequently, a stress test was performed and 30.93 mci of TC-99m Sestamibi injected IV at peak stress. After 30-60 minutes, gated SPECT images were"
for match in re.finditer(pattern_mci, string_2):
    s = match.start()
    e = match.end()
    print ('String match "%s" at %d:%d' % (string_2[s:e], s, e))

In [ ]:
mci_m = p2[p2["contains_mci"] != "[]"]
mci_m

In [ ]:
mci_m.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Edge_Cases\Keywords\apoe_1_mci.csv")

In [ ]:
p2[p2["merged_regex_match"].str.len() == 0]

In [ ]:
p2.head(10)

In [ ]:
string = str(p1["merged_regex_sent"][5]) + " mCi. "
print(string)
p = re.compile(r"(?i:)\bmemory\b|(?i:)\bno\b|(?-i:)\bOriented\b|(?-i:)\bMCI\b")

# p2 = re.compile(r"(?i:)\bMOCA\b")
# regexes = [p, p2]
# p_combined = re.compile(r'|'.join(x.pattern for x in regexes))
#print(p_combined)
#print(p)
for match in re.finditer(p_all , string):
    s = match.start()
    e = match.end()
    print ('String match "%s" at %d:%d' % (string[s:e], s, e))

In [ ]:
p3 = p2[:500]

In [ ]:
matches = []
pattern = re.compile(r'\bcerebral\b|\bcerebellar\b', re.IGNORECASE)

for note in tqdm(p2["merged_regex_sent"]):
    try:
        m = list(set(re.findall(pattern, note)))
        m = list(set(map(str.lower, m)))
        matches.append(str(m))
    except:
        matches.append([])

p2["contains_cerebral_cerebellar"] = matches

In [ ]:
cerebral_cerebellar = p2[p2["contains_cerebral_cerebellar"] != "[]"]
cerebral_cerebellar = cerebral_cerebellar[:500]

In [ ]:
matches2 = []
matches3 = []
matches4 = []

pattern2 = re.compile(r'\bCDR\b', re.IGNORECASE)
pattern3 = re.compile(r'\bMMSE\b', re.IGNORECASE)
pattern4 = re.compile(r"\bMOCA\b", re.IGNORECASE)

for note in tqdm(p2["merged_regex_sent"]):
    try:
        m = list(set(re.findall(pattern2, note)))
        m = list(set(m))
        matches2.append(str(m))
    except:
        matches2.append([])

p2["contains_cdr"] = matches2

for note in tqdm(p2["merged_regex_sent"]):
    try:
        m = list(set(re.findall(pattern3, note)))
        m = list(set(m))
        matches3.append(str(m))
    except:
        matches3.append([])

p2["contains_mmse"] = matches3

for note in tqdm(p2["merged_regex_sent"]):
    try:
        m = list(set(re.findall(pattern4, note)))
        m = list(set(m))
        matches4.append(str(m))
    except:
        matches4.append([])

p2["contains_moca"] = matches4

In [ ]:
moca_pattern = re.compile(r'\bmoca\b|\bMoca\b')
matches5 = []
for note in tqdm(p2["merged_regex_sent"]):
    try:
        m = list(set(re.findall(moca_pattern, note)))
        m = list(set((m)))
        matches5.append(str(m))
    except:
        matches5.append([])
p2["lower_case_moca"] = matches5

In [ ]:
cdr_pattern = re.compile(r'\bcdr\b|\bCdr\b') 
matches6 = []
for note in tqdm(p2["merged_regex_sent"]):
    try:
        m = list(set(re.findall(cdr_pattern, note)))
        m = list(set((m)))
        matches6.append(str(m))
    except:
        matches6.append([])
p2["lower_case_cdr"] = matches6

In [ ]:
mmse_pattern = re.compile(r'\bmmse\b|\bMmse\b') 
matches7 = []
for note in tqdm(p2["merged_regex_sent"]):
    try:
        m = list(set(re.findall(mmse_pattern, note)))
        m = list(set((m)))
        matches7.append(str(m))
    except:
        matches7.append([])
p2["lower_case_mmse"] = matches7

In [ ]:
p2["lower_case_moca"].value_counts()

In [ ]:
p2["lower_case_cdr"].value_counts()

In [ ]:
p2["lower_case_mmse"].value_counts()

In [ ]:
cdr = p2[p2["contains_cdr"] != "[]"]
len(cdr)

In [ ]:
mmse = p2[p2["contains_mmse"] != "[]"]
mmse = mmse[:500]

In [ ]:
moca = p2[p2["contains_moca"] != "[]"]
moca = moca[:500]

In [ ]:
p2["contains_moca"].value_counts()

In [ ]:
moca_lowercase = p2[(p2["lower_case_moca"] == "[\'moca\']") | (p2["lower_case_moca"] == "[\'Moca\']")]
moca_lowercase

In [ ]:
moca_lowercase["merged_regex_sent"][32080]

In [ ]:
cdr_lowercase = p2[(p2["lower_case_cdr"] == "[\'cdr\']") | (p2["lower_case_cdr"] == "[\'Cdr\']")]
cdr_lowercase

In [ ]:
p3.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Edge_Cases\Keywords\apoe_1_500_matches.csv", index = False)

In [ ]:
cerebral_cerebellar.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_cerebral_cerebellar.csv", index = False)

In [ ]:
cdr.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Edge_Cases\Keywords\apoe_1_cdr.csv", index = False)
mmse.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Edge_Cases\Keywords\apoe_1_mmse.csv", index = False)
moca.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Edge_Cases\Keywords\apoe_1_moca.csv", index = False)

In [ ]:
moca_lowercase.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Edge_Cases\Keywords\apoe_1_moca_lowercase.csv", index = False)

In [ ]:
cdr_lowercase.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Edge_Cases\Keywords\apoe_1_cdr_lowercase.csv", index = False)

## Getting #  of Unique Encounters per Patient

In [ ]:
## getting number of unique encounters per patient
df = p2 
#pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_matches_updated_pruned.csv")

In [ ]:
len(df)

In [ ]:
len(df["EMPI"].unique())

In [ ]:
df = df.drop_duplicates(subset = 'PatientEncounterID', keep = "first")

In [ ]:
summary_stats_1 = df.groupby(['EMPI'])["PatientEncounterID"].count().reset_index()
summary_stats_1

In [ ]:
summary_stats_1.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_summary_stats_7+8.csv")

In [ ]:
s1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_summary_stats_updated.csv")
# s2 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\2\apoe_2_summary_stats_updated.csv")
# s3 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\3\apoe_3_summary_stats_updated.csv")
# s4 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\4\apoe_4_summary_stats_updated.csv")
# s5 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\5\apoe_5_summary_stats_updated.csv")

In [ ]:
len(s1)

In [ ]:
len(s2)

In [ ]:
len(s3)

In [ ]:
len(s4)

In [ ]:
len(s5)

In [ ]:
2778+3335+3499+3431+3918

In [ ]:
frames = [s1, s2, s3, s4, s5]
total = pd.concat(frames)

In [ ]:
total.reset_index(drop = True)

In [ ]:
total.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\total_summary_stats_updated_7_6_2021.csv")

In [ ]:
subset = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_matches_updated_pruned.csv")

In [ ]:
subset = subset.iloc[0:5000]
subset

In [ ]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\Sentence-Analysis\EDA\Getting_Data\keywords.csv")

In [ ]:
regex.head()

In [ ]:
l = []
for note in tqdm(subset["merged_regex_sent"]):
    try:
        m = list(set(re.findall(re.compile("|".join(regex["REGEX"]), re.IGNORECASE), note)))
        m = list(set(map(str.lower, m)))
        l.append(m)
    except:
        l.append([])

subset["merged_regex_match"] = l

In [ ]:
a = ["cognition", "Cognition"]
a = list(set(map(str.lower, a)))
a

In [ ]:
subset.head()

In [ ]:
subset.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_matches_updated_v2_snippet.csv")

## Cleaning up Similar Encounters on Patient Level

In [ ]:
df = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\4\apoe_4_matches_updated_pruned.csv")

In [ ]:
df.head()

In [ ]:
# total_merged_sequences = []

# for i in tqdm(unique_empi):
#     merged_sequences = df[df["EMPI"] == i]["merged_regex_sent"].values
#     #print("MERGED: ", merged_sequences, "\n")

#     counter = 0
#     for j in merged_sequences:
# #         choices = []
        
# #         for k in merged_sequences:
# #             if (k != j):
# #                 choices.append(k)
# #         choices = merged_sequences[0:counter]
# #         choices += merged_sequences[counter+1:]
#         #print("\n------------------")
#         choices = list(merged_sequences[0:counter]) + list(merged_sequences[counter+1:])
#         result = get_match(j, choices)
#         score = result[0][1]
#         txt = result[0][0]
        
#         if (score >= 90):
#             # 90 is duplicate string threshold
#             idx = df[df["merged_regex_sent"] == str(txt)].index[0]
#             df.drop([idx])
        
#         counter += 1

In [ ]:
len(df)

In [ ]:
df["PatientEncounterID"].nunique()

In [ ]:
df["PatientEncounterID"].isna().sum()

In [ ]:
df = df.drop_duplicates(subset='merged_regex_sent', keep="first")

In [ ]:
len(df)

In [ ]:
dem1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\BioBank\sd587_03212118253982866_6375194824947826181_Dem.csv")

## Data Visualization

In [ ]:
updated = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\4\apoe_4_matches_updated.csv")

In [ ]:
len(updated)

In [ ]:
x = ((7509, 7518), (7735, 7741), (7745, 7754), (7858, 7862))
y = []
for i in range(len(x)):
    a = x[i][0]
    b = x[i][1]
    y.append((a,b))
print(str(y))
z = len(p1[(p1["regex_location"] == str(y)) & (p1["PatientEncounterID"] == int(encounters[21]))]['NoteTXT'].values[1])
z

In [ ]:
x = [(181, 390),
  (479, 688),
  (2048, 2269),
  (6184, 6390),
  (6584, 6821),
  (7450, 7659),
  (7716, 7932),
  (9864, 10072),
  (11192, 11648),
  (12118, 12627)]
y = []
for i in range(len(x)):
    a = x[i][0]
    b = x[i][1]
    y.append((a,b))
print(y)

In [ ]:
len(p1[(p1["regex_location"] == "[" +str(subset_loc[1]) + "]") & (p1["PatientEncounterID"] == int(encounters[1]))]['NoteTXT'].values[0])

In [ ]:
'at sleeping with his legs elevated on a pillow at night does help his leg swelling. Recommended Triamcinolone 0.1% ointment every morning and Mupirocin ointment at night, risks/benefits reviewed. Path'